In [ ]:
!pip install openrouteservice
!pip install fontawesomefree==6.1.1 

     |████████████████████████████████| 26.1 MB 1.0 MB/s 


In [ ]:
import folium
from openrouteservice import client

api_key = '5b3ce3597851110001cf6248bdfcb2470b4d42498141831f4d8f4703'  # Provide your personal API key
ors = client.Client(key=api_key)

# Inicializa el mapa
mapa = folium.Map(tiles='Stamen Toner', location=([39.46832, -6.3793]), zoom_start=14)

# Lista de preferencias del usuario
preferencias = {"supermarket" : 0,
                "platform" : 0,     #Paradas de bus
                "bus_station" : 0,
                "hairdresser" : 0,
                "pub" : 0,
                "bar" : 0,
                "fast-food" : 1,
                "pharmacy" : 0,                
                "fitness_centre" : 0,
                "park" : 0,
                "bank" : 0,
                "hospital" : 0,
                "university" : 0
                }

# Lista de pisos
pisos = {'Piso 1': {'location': [39.46925, -6.37852]},
         #'Piso 2': {'location': [39.4687,-6.3807]},
         #'Piso 3': {'location': [39.4671,-6.3799]},
         #'Piso 4': {'location': [39.4745,-6.3509]}
        }

# Parametros de las zonas de al rededor de los pisos
params_iso = {'profile': 'foot-walking',
              'range': [20 * 60],  # Limites de la zona a los que llegas en X minutos
             }

# Common request parameters
parametros_pi = {'request': 'pois',
                 'sortby': 'distance'
                }

# POI categories according to
# https://giscience.github.io/openrouteservice/documentation/Places.html
categorias_pi = { 'supermarket': [518],
                  'platform' : [607],
                  'bus_station' : [587],
                  'hairdresser': [395],
                  'pub' : [569],
                  'bar' : [561],
                  'fast-food' : [566],
                  'pharmacy' : [208],
                  'fitness_centre' : [288],
                  'park' : [272],
                  'bank' : [192],
                  'hospital' : [206],
                  'university' : [157]
                 }

# Set up dict for font-awesome
iconos = {'supermarket': 'shopping-cart',
          'platform' : 'bus',
          'bus_station' : 'map-signs',
          'hairdresser': 'scissors',
          'pub' : 'music',
          'night-club' : 'music',
          'bar' : 'beer',
          'fast-food' : 'cutlery',
          'pharmacy' : 'medkit',
          'fitness_centre' : 'futbol-o',
          'park' : 'leaf',
          'bank' : 'eur',
          'hospital' : 'hospital-o',
          'university' : 'graduation-cap'
          }

# Set up common request parameters
parametros_ruta = {'profile': 'foot-walking',
                   'format_out': 'geojson',
                   'geometry': 'true',
                   'format': 'geojson',
                   'instructions': 'false',
                   'radiuses' : 10000
                  }

In [ ]:
for nombre, piso in pisos.items():
    # Iconos Pisos
    folium.Marker(piso['location'], 
                  icon=folium.Icon(color='green',
                                    icon_color='#0049ff',
                                    icon='home',
                                    prefix='fa',
                                  ),
                  popup=nombre).add_to(mapa)
      
    # Zona piso
    params_iso['locations'] = [list(reversed(piso['location']))]  # Add apartment coords to request parameters
    piso['iso'] = ors.isochrones(**params_iso)  # Perform isochrone request
    folium.features.GeoJson(piso['iso']).add_to(mapa)  # Add GeoJson to map


for nombre, piso in pisos.items():
    piso['categories'] = dict()  # Store in pois dict for easier retrieval
    parametros_pi['geojson'] = piso['iso']['features'][0]['geometry']
    print("\n{} apartment".format(nombre))

    # Elimina del diccionario de puntos de interes aquellos que no quiere el usuario
    for categoria, pref in preferencias.items():
        if pref == 0:
            del categorias_pi[categoria]        
    
    for tipo, categoria in categorias_pi.items():
        parametros_pi['filter_category_ids'] = categoria
        piso['categories'][tipo] = dict()
        piso['categories'][tipo]['geojson'] = ors.places(**parametros_pi)['features']  # Actual POI request
        print(f"\t{tipo}: {len(piso['categories'][tipo]['geojson'])}")

    for cat, pois in piso['categories'].items():
        pois['durations'] = []
        for poi in pois['geojson']:
            poi_coords = poi['geometry']['coordinates']

            # Perform actual request
            parametros_ruta['coordinates'] = [piso['location'],
                                           list(reversed(poi_coords))
                                           ]
            json_route = ors.directions(**parametros_ruta)

            folium.features.GeoJson(json_route).add_to(mapa)
            folium.map.Marker(list(reversed(poi_coords)),
                              icon=folium.Icon(color='white',
                                               icon_color='#1a1aff',
                                               icon=iconos[cat],
                                               prefix='fa'
                                               )
                              ).add_to(mapa)

            poi_duration = json_route['features'][0]['properties']['summary']['duration']
            pois['durations'].append(poi_duration)  # Record durations of routes
            
mapa


Piso 1 apartment
	fast-food: 8
